In [84]:
# part 1 and 2
import re
from heapq import heapify, heappush, heappop

# lines = open('inputs/input_16_test.txt','r').read().splitlines()
lines = open('inputs/input_16.txt','r').read().splitlines()

valves = {}
for line in lines:
    doubles = [x for x in re.findall(r'([A-Z]{2})', line)]
    a = re.search(r'\d+', line) # find integer 
    flow = int(a.group(0))
    valves[doubles[0]] = flow,doubles[1:]

# _ = [print(x,valves[x]) for x in valves]


In [89]:
# part 1
def move(dP, clock, loc, openset):
    for newloc in valves[loc][1]:
        heappush(todo,(dP, clock, newloc, openset))
                
def openvalve(dP, clock, loc, openset):
    if loc not in openset and valves[loc][0] > 0:
        dP -= valves[loc][0]*(30-clock)
        heappush(todo, (dP, clock, loc, (*openset,loc)))

# print(valves)
# completed = [] # heap of completed states
dP_biggest_drop = 0
visited = {}  # dictionary of cache of visited, key: loc,openset, value: clock
todo = [(0,0,'AA',())]  # starting state
clock = 0
cnt = 0
while todo != []:
    cnt += 1
    dP, clock, loc, openset = heappop(todo)
    # if cnt % 1e7 == 0:
    #     print(f'cnt = {cnt}, current pop = {dP, clock, loc, openset}')
    if (loc,openset) in visited and clock > visited[(loc,openset)]:
        continue # move on to next pop
    if dP < dP_biggest_drop:
        dP_biggest_drop = dP
        clock_best = clock
        # print(f'Best so far: {dP, clock, loc, openset}')
    visited[(loc,openset)] = clock
    clock += 1
    if clock <= 29:
        if valves[loc][0] > 0:
            openvalve(dP, clock, loc, openset)
        move(dP, clock, loc, openset)

print(f'Best so far: {dP_biggest_drop, clock_best}')
print(f'cnt = {cnt}, current pop = {dP, clock, loc, openset}')
# test case: Best so far: (-1651, 24, 'CC', ('DD', 'BB', 'JJ', 'HH', 'EE', 'CC'))
# 16 sec, cnt = 12171829, Best so far: (-2114, 28, 'PW', ('AY', 'GC', 'WT', 'SY', 'VR', 'CB', 'PW'))

Best so far: (-2114, 28)
cnt = 12171829, current pop = (0, 14, 'YA', ())


In [90]:
# part 2
import itertools
import copy

def action(dP, clock, loc, openset):
    options_A = ('open', *valves[loc[0]][1])
    options_B = ('open', *valves[loc[1]][1])
    for r in itertools.product(options_A, options_B):
        newloc_A, newloc_B = loc
        dPnew = dP
        go_flag = True  # assume we can push todo unless one combo is open and can't open
        # if r[0] != r[1]:
        #     continue

        if r[0] != 'open' and r[1] != 'open': #and (dP, clock, (r[0], r[1]), openset) not in todo:
            newloc_A, newloc_B = r
            newset = openset

        elif r[0] == 'open' and r[1] == 'open' and loc[0] not in openset and valves[loc[0]][0] > 0 and loc[1] not in openset and valves[loc[1]][0] > 0:
            dPnew -= valves[loc[0]][0]*(TOTAL_TIME-clock)
            newset = (*openset,loc[0])
            if loc[1] != loc[0]:
                dPnew -= valves[loc[1]][0]*(TOTAL_TIME-clock)
                newset = (*newset,loc[1])

        elif r[0] == 'open' and r[1] != 'open' and loc[0] not in openset and valves[loc[0]][0] > 0:
            dPnew -= valves[loc[0]][0]*(TOTAL_TIME-clock)
            newset = (*openset,loc[0])
            newloc_B = r[1]

        elif r[0] != 'open' and r[1] == 'open' and loc[1] not in openset and valves[loc[1]][0] > 0:
            dPnew -= valves[loc[1]][0]*(TOTAL_TIME-clock)
            newset = (*openset,loc[1])
            newloc_A = r[0]

        else:
            go_flag = False

        if go_flag and newloc_A != newloc_B: # for part 1: newloc_A == newloc_B: and set TOTAL_TIME = 30
            if ( ( ((newloc_A,newloc_B),newset) not in visited
                or ( ((newloc_A,newloc_B),newset) in visited and clock < visited[((newloc_A,newloc_B),newset)] ) )
                and ((  ((newloc_B,newloc_A),newset) not in visited
                or ( ((newloc_B,newloc_A),newset) in visited and clock < visited[((newloc_B,newloc_A),newset)] )  ))):
                    
                    if (dPnew, clock, (newloc_A,newloc_B) , newset) not in todo:
                        heappush(todo,(dPnew, clock, (newloc_A,newloc_B) , newset))

def theoretically_more_dP(dP, clock, loc, openset):
    # check for what best possible pressure is possible given the current state
    closed_valves = []
    for x in valves:
        if x not in openset:
            # print(f'{x} not in openset')
            closed_valves.append(valves[x][0])
    closed_valves.sort(reverse=True)
    clock_list = [x for x in range(TOTAL_TIME-clock, 0, -2)]  # drop the clock by 2 per valve
    # print(clocks_list,len(clocks_list))
    extra_dP = 0
    i = 0
    while i < len(clock_list) and closed_valves[i] > 0:
        extra_dP += clock_list[i]*closed_valves[i]
        i += 1
    # print(f'extra_dP = {extra_dP}')
    return extra_dP

TOTAL_TIME = 26  # part 1: 30, # part 2: 26
# print(valves)
# completed = [] # heap of completed states
dP_biggest_drop = 0
visited = {}  # dictionary of cache of visited, key: loc,openset, value: clock
todo = [(0,0,('AA','AA'),())]  # starting state: (dP, clock, loc, openset)
clock = 0
cnt = 0
while todo != []:
    cnt += 1
    dP, clock, loc, openset = heappop(todo)
    visited[(loc,openset)] = clock
    if cnt % 1e7 == 0:
        print(f'cnt = {cnt}, current pop = {dP, clock, loc, openset}')
    if (loc,openset) in visited and clock > visited[(loc,openset)]:
        continue # move on to next pop
    if dP < dP_biggest_drop:
        dP_biggest_drop = dP
        best_clock = clock
        # print(f'Best so far: {dP, clock, loc, openset}')
    if dP - theoretically_more_dP(dP, clock, loc, openset) > dP_biggest_drop:
        # print(f'dP = {dP}, dP_biggest_drop = {dP_biggest_drop}')
        continue # dead branch, can't beat the best
    clock += 1
    if clock < TOTAL_TIME:
        action(dP, clock, loc, openset)

print(f'Best so far: {dP_biggest_drop} clock = {clock_best}')
print(f'cnt = {cnt}, current pop = {dP, clock, loc, openset}')

# part 1: .9 sec
# Best so far: (-2114, 28, ('PW', 'PW'), ('AY', 'GC', 'WT', 'SY', 'VR', 'CB', 'PW'))
# cnt = 2414830, current pop = (0, 14, ('YA', 'YA'), ())

# solved: 14 sec
# Best so far: (-2666, 24, ('XI', 'JS'), ('AY', 'CB', 'GC', 'VR', 'QT', 'WT', 'VG', 'SY', 'RT', 'QO', 'XI', 'JS'))
# cnt = 432096, current pop = (0, 5, ('ZX', 'YN'), ())


Best so far: -2666 clock = 28
cnt = 432096, current pop = (0, 5, ('ZX', 'YN'), ())
